In [1]:
import random
import numpy as np
import enum
import time
from termcolor import colored

In [2]:
class LiftActions(enum.Enum):
    SKIP = 0
    UP1 = 1
    UP2 = 2
    DOWN1 = 3
    DOWN2 = 4
    OPEN1 = 5
    OPEN2 = 6
    CLOSE1 = 7
    CLOSE2 = 8

    START = 9
    FINISH = 10

class LiftsPassState(enum.Enum):
    NONE = 0
    IN1 = 1
    IN2 = 2

In [3]:
class TwoLifts:

    def __init__(self, floors: int, start_state: tuple, print_movments: bool = False):
        self.floors = floors
        self.print_movments = print_movments
        # Состояние: этаж 1 лифта, этаж второго лифта, (0 - нет пассажиров, 1 - они в 1 лифте, 2 - они во 2 лифте)
        self.state = (*start_state, 0)
        
        self.states_dca = {}

        self.move_count = 0
        self.all_move_count = 0

        self.move_command = [None, None]

        self.queue_symb = [None]
        self.current_symb = None

        self.actions = {
            LiftActions.SKIP: None,
            LiftActions.UP1: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] + 1}, {self.state[1]}]") if self.print_movments else None),
            LiftActions.DOWN1: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] - 1}, {self.state[1]}]") if self.print_movments else None),
            LiftActions.OPEN1: lambda: (print("Лифт 1: открыт") if self.print_movments else None),
            LiftActions.CLOSE1: lambda: (print("Лифт 1: закрыт") if self.print_movments else None),
            
            LiftActions.UP2: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] + 1}]") if self.print_movments else None),
            LiftActions.DOWN2: lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] - 1}]") if self.print_movments else None),
            LiftActions.OPEN2: lambda: (print("Лифт 2: открыт") if self.print_movments else None),
            LiftActions.CLOSE2: lambda: (print("Лифт 2: закрыт") if self.print_movments else None),

            LiftActions.START: lambda: (self.next_symb()),
            LiftActions.FINISH: lambda: (print(colored(f"Для вызова ({self.move_command}) было затрачено {self.move_count} операций.", "light_green")), self.plus_all_move_count(), self.next_symb())

        }
    def plus_move_count(self):
        self.move_count += 1
    def plus_all_move_count(self):
        self.all_move_count += self.move_count
        self.move_count = 0

    def next_symb(self):
        self.current_symb = self.queue_symb.pop(0)
        self.move_command.pop(0)
        self.move_command.append(self.current_symb)
        # print("append", self.current_symb)

    def run(self, list_commands):
        self.queue_symb = [item for sublist in list_commands for item in sublist] + [None]
        self.next_symb()
        # print(self.state, self.current_symb)
        while (self.current_symb):
            # time.sleep(0.2)
            # print(self.state, self.current_symb)
            next_item = self.states_dca[self.state][self.current_symb]
            next_item["action"]()
            self.state = next_item["state"]
        print(colored(f"Всего операций {self.all_move_count}", "light_green"))

            

    def generate_dca(self):
        for floor1 in range(1, self.floors + 1):
            for floor2 in range(1, self.floors + 1):
                for pass_state in range(0, 3):
                    self.states_dca[floor1, floor2, pass_state] = {}
                    for next_floor in range(1, self.floors + 1):
                        if abs(floor1 - next_floor) <= abs(floor2 - next_floor) and pass_state != 2:
                            if floor1 < next_floor:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1 + 1, floor2, pass_state),
                                    "action": self.actions[LiftActions.UP1]
                                }
                            elif floor1 > next_floor:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1 - 1, floor2, pass_state),
                                    "action": self.actions[LiftActions.DOWN1]
                                }
                            else:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1, floor2, 0 if pass_state == 1 else 1),
                                    "action": (lambda: (self.actions[LiftActions.OPEN1](), self.actions[LiftActions.CLOSE1](), self.actions[LiftActions.FINISH]())) if pass_state != 0 
                                        else (lambda: (self.actions[LiftActions.OPEN1](), self.actions[LiftActions.CLOSE1](), self.actions[LiftActions.START]())),
                                }
                        else: 
                            if floor2 < next_floor:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1, floor2 + 1, pass_state),
                                    "action": self.actions[LiftActions.UP2]
                                }
                            elif floor2 > next_floor:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1, floor2 - 1, pass_state),
                                    "action": self.actions[LiftActions.DOWN2]
                                }
                            else:
                                self.states_dca[floor1, floor2, pass_state][next_floor] = {
                                    "state": (floor1, floor2, 0 if pass_state == 2 else 2),
                                    "action": (lambda: (self.actions[LiftActions.OPEN2](), self.actions[LiftActions.CLOSE2](), self.actions[LiftActions.FINISH]())) if pass_state != 0 
                                        else (lambda: (self.actions[LiftActions.OPEN2](), self.actions[LiftActions.CLOSE2](), self.actions[LiftActions.START]())),
                                }
                                

In [4]:
a = np.array([1, 2, 3])
a.astype(int)

array([1, 2, 3])

In [7]:
# floors = list(range(1, random.randint(5, 21)))
floors = list(range(1, 16))

print(floors[-1])

# actions_count = random.randint(5, 10)
# actions = [np.random.choice(floors, 2).tolist() for i in range(actions_count)]
actions = [[2, 10], [6, 8], [2, 7], [1, 3], [11, 7], [3, 7], [3, 11], [9, 12], [11, 2]]

print(actions)

# start_state = tuple(map(lambda x: int(x), np.random.choice(floors, 2)))
start_state = (9, 6)

print(start_state)


15
[[2, 10], [6, 8], [2, 7], [1, 3], [11, 7], [3, 7], [3, 11], [9, 12], [11, 2]]
(9, 6)


In [8]:
l = TwoLifts(floors=floors[-1], start_state=start_state, print_movments=True)
l.generate_dca()
l.run(actions)

Лифты: [9, 6]->[9, 5]
Лифты: [9, 5]->[9, 4]
Лифты: [9, 4]->[9, 3]
Лифты: [9, 3]->[9, 2]
Лифт 2: открыт
Лифт 2: закрыт
Лифты: [9, 2]->[9, 3]
Лифты: [9, 3]->[9, 4]
Лифты: [9, 4]->[9, 5]
Лифты: [9, 5]->[9, 6]
Лифты: [9, 6]->[9, 7]
Лифты: [9, 7]->[9, 8]
Лифты: [9, 8]->[9, 9]
Лифты: [9, 9]->[9, 10]
Лифт 2: открыт
Лифт 2: закрыт
Для вызова ([2, 10]) было затрачено 12 операций.
Лифты: [9, 10]->[8, 10]
Лифты: [8, 10]->[7, 10]
Лифты: [7, 10]->[6, 10]
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [6, 10]->[7, 10]
Лифты: [7, 10]->[8, 10]
Лифт 1: открыт
Лифт 1: закрыт
Для вызова ([6, 8]) было затрачено 5 операций.
Лифты: [8, 10]->[7, 10]
Лифты: [7, 10]->[6, 10]
Лифты: [6, 10]->[5, 10]
Лифты: [5, 10]->[4, 10]
Лифты: [4, 10]->[3, 10]
Лифты: [3, 10]->[2, 10]
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [2, 10]->[2, 9]
Лифты: [2, 9]->[2, 8]
Лифты: [2, 8]->[2, 7]
Лифт 2: открыт
Лифт 2: закрыт
Для вызова ([2, 7]) было затрачено 9 операций.
Лифты: [2, 7]->[2, 6]
Лифты: [2, 6]->[2, 5]
Лифты: [2, 5]->[2, 4]
Лифты: [2, 4]->